![](./resources/System_v1_training_header.png)

This notebook contains a demonstration on how to train custom crop type models based on your own reference data and how to apply the resulting model to generate a custom crop type map.

# Content

- [Before you start](#before-you-start)
- [1. Define region of interest](#1.-Define-a-region-of-interest)
- [2. Check public in-situ reference data](#2.-Check-public-in-situ-reference-data)
- [3. Prepare own reference data](#3.-Prepare-own-reference-data)
- [4. Extract required model inputs](#4.-Extract-required-model-inputs)
- [5. Train custom classification model](#5.-Train-custom-classification-model)
- [6. Generate a map](#6.-Generate-a-map)

# Before you start

In order to run this notebook, you need to create an account on:

- The Copernicus Data Space Ecosystem (CDSE)
--> by completing the form [HERE](https://identity.dataspace.copernicus.eu/auth/realms/CDSE/login-actions/registration?client_id=cdse-public&tab_id=eRKGqDvoYI0)

- VITO's Terrascope platform
--> by completing the form [HERE](https://sso.terrascope.be/auth/realms/terrascope/login-actions/registration?client_id=drupal-terrascope&tab_id=irBzckp2aDo)

In [2]:
# First we import the necessary modules to run this notebook

import pandas as pd
import geopandas as gpd
import requests
from pathlib import Path
from shapely.geometry import shape, Polygon
import xarray as xr

import openeo
from openeo_gfmap import BoundingBoxExtent, TemporalContext
from openeo_gfmap.backend import Backend, BackendContext

import sys
sys.path.append('/home/jovyan/worldcereal-classification/src')
# sys.path.append('/home/cbutsko/Desktop/worldcereal-classification/src')

from worldcereal.utils.map import (get_ui_map, _latlon_to_utm)
from worldcereal.utils.refdata import _to_points
from worldcereal.utils.wrapper import run_inference

RDM_API = 'https://ewoc-rdm-api.iiasa.ac.at'

# 1. Define a region of interest

When running the code snippet below, an interactive map will be visualized.
Click the Rectangle button on the left hand side of the map to start drawing your region of interest.
When finished, execute the second cell to store the coordinates of your region of interest. 

In [3]:
m, dc = get_ui_map()
m

Map(center=[51.1872, 5.1154], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [4]:
# retrieve bounding box from drawn rectangle
obj = dc.last_draw
if obj.get('geometry') is not None:
    poly = Polygon(shape(obj.get('geometry')))
    bbox = poly.bounds
else:
    raise ValueError('Please first draw a rectangle '
                     'on the map before proceeding.')
print(f'Your area of interest: {bbox}')

Your area of interest: (4.511536, 51.12835, 5.24762, 51.312417)


# 2. Check public in situ reference data

Here we do a series of requests to the RDM API to retrieve the collections and samples overlapping our bbox...

‼ The following snippet does not query the RDM API, but parquet file on Cloudferro bucket with Phase I extractions

In [29]:
# additional imports

import json
import numpy as np
import duckdb
import ipywidgets as widgets
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

# added presto path the same way world cereal is added. right now it needs the branch inference-extension-to-parquet
# should be changed to master once the branch is merged 
presto_repo_path = "/home/cbutsko/Desktop/presto-worldcereal/"
sys.path.append(presto_repo_path)
from presto.inference import get_presto_features, process_parquet
from presto.presto import Presto
from presto.dataset import WorldCerealLabelledDataset

from presto.utils import DEFAULT_SEED, device

# this path is hardcoded now 😥
data_dir = "/home/cbutsko/Desktop/worldcereal-classification/notebooks/resources"
with open(f"{data_dir}/croptype_classes.json") as f:
    CLASS_MAPPINGS = json.load(f)

In [8]:
# not sure where to put these functions 😥
def get_encodings_targets(
    df: pd.DataFrame, presto_model, batch_size: int = 1024
) -> [np.ndarray, np.ndarray]:

    tds = WorldCerealLabelledDataset(df, target_function=lambda xx: xx["custom_class"])
    tdl = DataLoader(tds, batch_size=batch_size, shuffle=False)

    encoding_list, targets = [], []

    for x, y, dw, latlons, month, variable_mask in tdl:
        x_f, dw_f, latlons_f, month_f, variable_mask_f = [
            t.to(device) for t in (x, dw, latlons, month, variable_mask)
        ]
        input_d = {
            "x": x_f,
            "dynamic_world": dw_f.long(),
            "latlons": latlons_f,
            "mask": variable_mask_f,
            "month": month_f,
        }

        presto_model.eval()
        encodings = presto_model.encoder(**input_d).detach().numpy()

        encoding_list.append(encodings)
        targets.append(y)

    encodings_np = np.concatenate(encoding_list)
    targets = np.concatenate(targets)

    return encodings_np, targets


def map_croptypes(
    df: pd.DataFrame,
    downstream_classes="CROPTYPE9",
) -> pd.DataFrame:

    wc2ewoc_map = pd.read_csv(f"{data_dir}/wc2eurocrops_map.csv")
    wc2ewoc_map["ewoc_code"] = wc2ewoc_map["ewoc_code"].str.replace("-", "").astype(int)

    ewoc_map = pd.read_csv(f"{data_dir}/eurocrops_map_wcr_edition.csv")
    ewoc_map = ewoc_map[ewoc_map["ewoc_code"].notna()]
    ewoc_map["ewoc_code"] = ewoc_map["ewoc_code"].str.replace("-", "").astype(int)
    ewoc_map = ewoc_map.apply(lambda x: x[: x.last_valid_index()].ffill(), axis=1)
    ewoc_map.set_index("ewoc_code", inplace=True)

    df["CROPTYPE_LABEL"].replace(0, np.nan, inplace=True)
    df["CROPTYPE_LABEL"].fillna(df["LANDCOVER_LABEL"], inplace=True)

    df["ewoc_code"] = df["CROPTYPE_LABEL"].map(
        wc2ewoc_map.set_index("croptype")["ewoc_code"]
    )
    df["landcover_name"] = df["ewoc_code"].map(ewoc_map["landcover_name"])
    df["cropgroup_name"] = df["ewoc_code"].map(ewoc_map["cropgroup_name"])
    df["croptype_name"] = df["ewoc_code"].map(ewoc_map["croptype_name"])

    df["downstream_class"] = df["ewoc_code"].map(
        {int(k): v for k, v in CLASS_MAPPINGS[downstream_classes].items()}
    )

    return df

In [9]:
db = duckdb.connect()
db.sql('INSTALL spatial')
db.load_extension("spatial")

parquet_path = "s3://geoparquet/worldcereal_extractions_phase1/*/*.parquet"

# only querying the croptype data here
public_df_raw = db.sql(
    f"""
set s3_endpoint='s3.waw3-1.cloudferro.com';
select *
from read_parquet('{parquet_path}', hive_partitioning = 1) original_data
where st_within(ST_Point(original_data.lon, original_data.lat), ST_GeomFromText('{poly.wkt}'))
and original_data.CROPTYPE_LABEL not in (0, 991, 7900, 9900, 9998, 1910, 1900, 1920, 1000, 11, 9910, 6212, 7920, 9520, 3400, 3900, 4390, 4000, 4300)
"""
).df()

In [10]:
public_df_raw.shape

(75152, 32)

In [11]:
public_df = process_parquet(public_df_raw)
public_df = map_croptypes(public_df)

# 3.Prepare own reference data

Include some guidelines on how to upload user dataset to RDM (using the UI) and requesting those user samples through the API.

In [12]:
merged_df = public_df.copy()

### 3a. Select desired crops for prediction

Crops with ticked checkboxes will be included in the prediction. All the crops that are not selected will be grouped under the "other_crop" category.

In [16]:
potential_classes

croptype_name  count
0  pasture_meadows   2324
1            maize   1861
2         potatoes    208
3  permanent_crops    183

In [20]:
samples_threshold = 100
potential_classes = merged_df['croptype_name'].value_counts().reset_index()
potential_classes = potential_classes[potential_classes['count']>samples_threshold]

checkbox_widgets = [widgets.Checkbox(value=False, description=f"{row['croptype_name']} ({row['count']} samples)") for ii,row in potential_classes.iterrows()]
widgets.VBox(checkbox_widgets, layout=widgets.Layout(width='50%', display='inline-flex', flex_flow='row wrap'))

In [22]:
seleted_crops = [
    checkbox.description.split(" ")[0]
    for checkbox in checkbox_widgets
    if checkbox.value
]
merged_df["custom_class"] = "other"
merged_df.loc[
    merged_df["croptype_name"].isin(seleted_crops), "custom_class"
] = merged_df["croptype_name"]

In [23]:
merged_df["custom_class"].value_counts()

custom_class
pasture_meadows    2324
maize              1861
other               321
potatoes            208
permanent_crops     183
Name: count, dtype: int64

# 4. Extract required model inputs

Here we launch point extractions for all samples intersecting our bbox resulting in a set of parquet files.

We collect all these inputs and prepare presto features for each sample.

In [37]:
trn_df, val_df = train_test_split(
    merged_df,
    stratify=merged_df["custom_class"],
    test_size=0.3,
    random_state=DEFAULT_SEED,
)

In [39]:
presto_model_path = f"{data_dir}/presto-ss-wc-ft-ct-30D_test.pt"
presto_model = Presto.load_pretrained(model_path=presto_model_path, strict=False)

train_encodings, train_targets = get_encodings_targets(trn_df, presto_model, batch_size=256)
val_encodings, val_targets = get_encodings_targets(val_df, presto_model, batch_size=256)

# 5. Train custom classification model
We train a catboost model and upload this model to artifactory.

In [40]:
if np.unique(train_targets).shape[0] > 1:
    eval_metric = "TotalF1"
else:
    eval_metric = "F1"

class_weights = compute_class_weight(
    class_weight="balanced", classes=np.unique(train_targets), y=train_targets)

custom_downstream_model = CatBoostClassifier(
    iterations=8000,
    depth=8,
    learning_rate=0.05,
    early_stopping_rounds=50,
    # l2_leaf_reg=30,
    colsample_bylevel=0.9,
    l2_leaf_reg=6,
    eval_metric=eval_metric,
    random_state=DEFAULT_SEED,
    # class_weights=class_weights,
    verbose=25,
    class_names=np.unique(train_targets),
)

custom_downstream_model.fit(
    train_encodings, train_targets, eval_set=Pool(val_encodings, val_targets)
)

pred = custom_downstream_model.predict(val_encodings).flatten()

0:	learn: 0.6550097	test: 0.6250140	best: 0.6250140 (0)	total: 160ms	remaining: 21m 21s
25:	learn: 0.6855338	test: 0.6656578	best: 0.6656578 (25)	total: 2.78s	remaining: 14m 14s
50:	learn: 0.7196221	test: 0.6831521	best: 0.6831521 (50)	total: 5.45s	remaining: 14m 9s
75:	learn: 0.7344926	test: 0.7032237	best: 0.7032237 (75)	total: 8.13s	remaining: 14m 8s
100:	learn: 0.7496629	test: 0.7054853	best: 0.7054853 (100)	total: 10.7s	remaining: 13m 59s
125:	learn: 0.7716846	test: 0.7173224	best: 0.7173224 (124)	total: 13.4s	remaining: 13m 59s
150:	learn: 0.7924791	test: 0.7287346	best: 0.7293823 (145)	total: 16.1s	remaining: 13m 56s
175:	learn: 0.8021751	test: 0.7394657	best: 0.7394662 (172)	total: 18.8s	remaining: 13m 56s
200:	learn: 0.8170485	test: 0.7414905	best: 0.7414905 (196)	total: 21.4s	remaining: 13m 51s
225:	learn: 0.8305947	test: 0.7475895	best: 0.7482350 (221)	total: 24.1s	remaining: 13m 48s
250:	learn: 0.8409549	test: 0.7539567	best: 0.7539567 (250)	total: 26.7s	remaining: 13m 45s


In [41]:
print(classification_report(val_targets, pred))

                 precision    recall  f1-score   support

          maize       0.79      0.85      0.82       559
          other       0.62      0.11      0.19        89
pasture_meadows       0.81      0.91      0.85       698
permanent_crops       0.79      0.27      0.41        55
       potatoes       0.80      0.53      0.64        62

       accuracy                           0.80      1463
      macro avg       0.76      0.53      0.58      1463
   weighted avg       0.79      0.80      0.77      1463



# 6. Generate a map

Using our custom model, we generate a map for our region of interest...